# 调用get_kw_
参数为：ref_file  为参考文献的文件
cooccurrence_csv   要将提取的关键知识保存到哪
processed_csv   要将已处理的参考文献保存到哪

In [23]:
import nltk
#nltk.download('punkt_tab')
#nltk.download('punkt')
from collections import Counter
import re
from itertools import combinations
from keybert import KeyLLM
import openai
from keybert.llm import OpenAI

def extract_keywords(abstract):
    prompt = """
    I have the following abstract:
    [DOCUMENT]
    
    Based on the Abstract above, extract the keywords that best describe the topic of the abstract.
    Make sure all extracted keywords or concepts appear verbatim in the text and do not exceed ten keywords.
    Use the following format separated by commas:
    <Knowledge>
    """

    #base_url = "https://api.openai-all.com/v1"
    #api_key = 'sk-I5JvAkB64SMc0pt_0CiD7gzFtYcj_qhD_FoIgWnCGlMijOLjBPc0h1H4zcE'
    
    # v-api
    base_url = "https://api.gpt.ge/v1/"
    api_key = "sk-uDpJ1lRpOwCLOyoZ15C57f7fA84643E9808e717bC30fB99b"
    #model = "gpt-4o"
    model = "gpt-3.5-turbo"
    #ollama
    base_url = "https://api.deepseek.com/v1/"
    api_key="sk-f1ab015adcc14d55bba3b579a5918415" # required, but unused
    model = "deepseek-chat"
    # Create your LLM
    client  = openai.OpenAI(base_url=base_url,
        # required but ignored
        api_key=api_key)
    
    #model="llama3.3:70b-instruct-q2_k"  gpt-4o
    llm = OpenAI(client,model=model,prompt=prompt,chat=True)
    # Load it in KeyLLM
    kw_model = KeyLLM(llm)
    keywords = kw_model.extract_keywords(abstract,check_vocab=True)
    kws = keywords[0]
    print(kws)
    return kws

def analyze_cooccurrence_standardized(sentences, keywords):
    co_occurrence_counter = Counter()
    num_sentences = len(sentences)
    
    # 预编译正则表达式模式
    keyword_patterns = {kw: re.compile(r'\b' + re.escape(kw.lower()) + r'\b') for kw in keywords}
    
    for i in range(num_sentences):
        # 确定窗口范围
        if i == 0:
            window_sentences = sentences[i:i + 2]  # 第一个句子，包含第1和第2句
        elif i == num_sentences - 1:
            window_sentences = sentences[i - 1:i + 1]  # 最后一个句子，包含倒数第2和第5句
        else:
            window_sentences = sentences[i - 1:i + 2]  # 中间句子，包含前一句、当前句、后一句
        
        window_text = ' '.join(window_sentences).lower()
        
        # 找出窗口中出现的关键词
        present_keywords = [kw for kw in keywords if keyword_patterns[kw].search(window_text)]
        
        # 统计所有可能的关键词对
        for pair in combinations(present_keywords, 2):
            sorted_pair = tuple(sorted(pair))
            co_occurrence_counter[sorted_pair] = 1
    
    return co_occurrence_counter

def liucheng(abstract):
    keyword_list = extract_keywords(abstract)
    sentences = nltk.sent_tokenize(abstract)
    rst = analyze_cooccurrence_standardized(sentences,keyword_list)
    return rst

In [12]:
abstract = """
Recent studies documented the importance of individuality and heterogeneity in care planning. In practice, varying behavioral responses are revealed in patients' care management (CM) records. However, today's care programs are structured around population-level evidence. What if care managers can take advantage of the revealed behavioral response for personalization? The goal of this study is thus to quantify behavioral response from CM records for informing individual-level intervention decisions. We present a Behavioral Response Inference Framework (BRIeF) for understanding differential behavioral responses that are key to effective care planning. We analyze CM records from a healthcare network over a 14-month period and obtain a set of 2,416 intervention-goal attainment records. Promising results demonstrate that the individual-level care planning strategies that are learned from practice by BRIeF, outperform population-level strategies, yielding significantly more accurate intervention recommendations for goal attainment. To our knowledge, this is the first study of learning practice-based evidence from CM records for care planning, suggesting that increased patient behavioral understanding could potentially benefit augmented intelligence for care management decision support.
"""
rst = extract_keywords(abstract)
print(rst)

['heterogeneity', 'care planning', 'behavioral response', 'personalization', 'intervention decisions', 'Behavioral Response Inference Framework (BRIeF)', 'healthcare network', 'intervention-goal attainment', 'practice-based evidence', 'augmented intelligence']
['heterogeneity', 'care planning', 'behavioral response', 'personalization', 'intervention decisions', 'Behavioral Response Inference Framework (BRIeF)', 'healthcare network', 'intervention-goal attainment', 'practice-based evidence', 'augmented intelligence']


In [12]:
# 加载已有的共现关系
def load_cooccurrence(csv_file):
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        co_occurrence_counter = Counter()
        #print(csv_file)
        for _, row in df.iterrows():
            kw1, kw2 = row['Keyword1'], row['Keyword2']
            if pd.notna(kw1) and pd.notna(kw2):
                pair = tuple(sorted([row['Keyword1'], row['Keyword2']]))
                co_occurrence_counter[pair] = row['Count']
        print(f"已加载共现关系，共有 {len(co_occurrence_counter)} 个关键词对。")
    else:
        co_occurrence_counter = Counter()
        print("未找到共现关系文件，初始化一个新的计数器。")
    return co_occurrence_counter

def save_cooccurrence(co_occurrence_counter, csv_file):
    data = {
        'Keyword1': [pair[0] for pair in co_occurrence_counter.keys()],
        'Keyword2': [pair[1] for pair in co_occurrence_counter.keys()],
        'Count': [count for count in co_occurrence_counter.values()]
    }
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index=False)
    print(f"共现关系已保存到 '{csv_file}'。")
def load_processed_titles(processed_csv):
    """
    从 CSV 文件加载已处理摘要的标题。

    Args:
        processed_csv (str): 存储已处理标题的 CSV 文件路径。

    Returns:
        set: 已处理标题的集合。
    """
    if os.path.exists(processed_csv):
        processed_df = pd.read_csv(processed_csv)
        processed_titles = set(processed_df['title'].tolist())
        print(f"已加载 {len(processed_titles)} 个已处理的摘要标题。")
    else:
        processed_titles = set()
        print("未找到已处理摘要记录文件，初始化一个新的记录。")
    return processed_titles
def save_processed_titles(new_titles, processed_csv):
    """
    将新增处理的标题保存到 CSV 文件。

    Args:
        new_titles (list): 新处理的标题列表。
        processed_csv (str): 存储已处理标题的 CSV 文件路径。
    """
    if os.path.exists(processed_csv):
        processed_df = pd.read_csv(processed_csv)
    else:
        processed_df = pd.DataFrame(columns=['title'])
    
    # 合并新处理的标题
    new_df = pd.DataFrame({'title': new_titles})
    processed_df = pd.concat([processed_df, new_df], ignore_index=True)
    processed_df.to_csv(processed_csv, index=False)
    print(f"已更新 {len(new_titles)} 个已处理的摘要标题。")


In [13]:
import pandas as pd
import os
def get_kw_(ref_file,cooccurrence_csv,processed_csv):
    # 加载已处理标题
    processed_titles = load_processed_titles(processed_csv)
    # 加载CSV文件
    df = pd.read_excel(ref_file)
    titles = df['ref_id'].dropna().tolist()
    abstracts = df['abstract'].dropna().tolist()
    # 加载共现关系
    co_occurrence_counter = load_cooccurrence(cooccurrence_csv)
    # 新处理的标题
    new_processed_titles = []
    for title, abstract in zip(titles, abstracts):
        if title in processed_titles:
            continue  # 跳过已处理的摘要
        #print(f'处理到{title}')
        # 睡眠3秒
        #print(abstract)
        if abstract == None or abstract == '':
            print(f"跳过标题 '{title}'，因为摘要为空。")
            continue
        co_occurrence_counter1 = liucheng(abstract)
        if co_occurrence_counter == None:
            co_occurrence_counter = co_occurrence_counter1
        else:
            co_occurrence_counter += co_occurrence_counter1
        # 记录新处理的标题
        new_processed_titles.append(title)
    # 更新已处理的标题记录
    if new_processed_titles:
        save_processed_titles(new_processed_titles, processed_csv)
    # 保存共现关系到 CSV 文件
    save_cooccurrence(co_occurrence_counter, cooccurrence_csv)
    print("处理完成。")
    

## 获得所有的共现关系

In [24]:
# 读取all_focus.xlsx
import pandas as pd
import time
df = pd.read_excel('E:/document/research/dr_research/code/eswa临时/未获奖_1.xlsx')


if 'co_occurrence_counter' not in df.columns:
    df['co_occurrence_counter'] = None
    
# for循环，得到index title和abstract
has_done = 12701
for index,row in df.iterrows():
    if pd.isna(row['id']) or pd.isna(row['abstract']):
        continue
        
    if index <= has_done:
        continue
    if index > 14591:
        break
    
    
    print(f'处理到第{index}个')
    '''
    try:
        get_kw_(ref_file=f'E:/document/research/dr_research/code/eswa临时/noaward_ref/{index}_ref.xlsx', cooccurrence_csv = f'E:/document/research/dr_research/code/eswa临时/noaward_ref/{index}_cooccurrence.xlsx',processed_csv= f'E:/document/research/dr_research/code/eswa临时/noaward_ref/{index}_processed.xlsx')
    except FileNotFoundError as e:
        print(e)
        continue
    '''
    # 判断索引为index，的co_occurrence_counter是否不为空，如果不为空就continue
    if pd.notna(df.loc[index,'co_occurrence_counter']): 
        continue
    else:
        # 得到abstract列的第一行
        abstract = df.loc[index,'abstract']
        # 判断abstract是否为空，或者为nan，如果是就continue 
        if pd.isna(abstract) or not abstract:
            continue
        # 设置2s休眠
        co_occurrence_counter = liucheng(abstract)
        # 在all_focus.xlsx中加入新，为co_occurrence_counter，然后报错格式为Counter的co_occurrence_counter
        df.loc[index,'co_occurrence_counter'] = str(co_occurrence_counter)
        df.to_excel('E:/document/research/dr_research/code/eswa临时/未获奖_1.xlsx',index=False)

处理到第12702个
['interdependent behavior', 'Cohort Intelligence', 'self supervise', 'adapt', 'improve', 'evolve', 'cohort behavior', 'test problems']
处理到第12703个
['mobile robots', 'non-holonomic properties', 'uncertainties', 'cascade control architecture', 'interval type-2 fuzzy PID controllers', 'tracking performance', 'PIONEER 3-DX mobile robot', 'conventional PID', 'type-1 fuzzy controllers']
处理到第12704个
['SEM', 'causality analysis', 'multiple path models', 'integration', 'OR', 'probability concepts', 'experiment', 'path models', 'usability']
处理到第12705个
['brain machine interface', 'electromyogram signals', 'electro-encephalographic signals', 'feature vector extractor', 'neural network estimator', 'reference trajectory generator', 'adaptive corrector', 'forearm motions']
处理到第12706个
['PSO', 'gait learning', 'humanoid robots', 'robotics simulator system', 'Open Dynamics Engine', 'ODE', 'center of mass', 'motion training platform', 'walk forward']
处理到第12707个
['non-linear discriminant analysis


处理遗漏的单篇论文方法

In [22]:
df = pd.read_excel('data1/all_A_papers.xlsx')
abstract = """
Although image restoration has achieved significant progress, its potential to assist object detectors in adverse imaging conditions lacks enough attention. It is reported that the existing image restoration methods cannot improve the object detector performance and sometimes even reduce the detection performance. To address the issue, we propose a targeted adversarial attack in the restoration procedure to boost object detection performance after restoration. Specifically, we present an ADAM-like adversarial attack to generate pseudo ground truth for restoration training. Resultant restored images are close to original sharp images, and at the same time, lead to better results of object detection. We conduct extensive experiments in image dehazing and low light enhancement and show the superiority of our method over conventional training and other domain adaptation and multi-task methods. The proposed pipeline can be applied to all restoration methods and detectors in both one-and two-stage.
"""
co_occurrence_counter = liucheng(abstract)
df.loc[1445,'co_occurrence_counter'] = str(co_occurrence_counter)
df.to_excel('data1/all_A_papers.xlsx',index=False)

['image restoration', 'object detectors', 'adversarial attack', 'restoration procedure', 'ADAM-like', 'pseudo ground truth', 'image dehazing', 'low light enhancement', 'domain adaptation', 'multi-task methods']


In [11]:
need_process = [16, 35, 40, 50, 53, 56, 66, 88, 133, 149, 184, 254, 260, 279, 366, 631, 751, 785, 890, 918, 1082, 1118, 1123, 1217, 1235, 1277, 1324]

In [12]:
import pandas as pd
df = pd.read_excel('data1/all_A_papers.xlsx')
import time
# 找到df中所有co_occurrence_counter为空的，如果为空就进行处理
for index in need_process:
    print(index)
    try:
        get_kw_(ref_file=f'data1/{index}_ref.csv', cooccurrence_csv = f'data1/{index}_cooccurrence.csv',processed_csv= f'data1/{index}_processed.csv')
    except FileNotFoundError as e:
        continue
    if pd.notna(df.loc[index,'co_occurrence_counter']):
        continue
    else:
        df1 = pd.read_csv(f'data1/{index}_paper.csv')
        abstract = df1['abstract'][0]
        if pd.isna(abstract) or not abstract:
            continue
        co_occurrence_counter = liucheng(abstract)
        df.loc[index,'co_occurrence_counter'] = str(co_occurrence_counter)
        df.to_excel('data1/all_A_papers.xlsx',index=False)

751
未找到已处理摘要记录文件，初始化一个新的记录。
未找到共现关系文件，初始化一个新的计数器。
['model evaluation', 'data-oriented', 'representation-oriented', 'process-oriented evaluation', 'rule induction', 'error estimation']
['average case analysis', 'induction algorithms', 'monotone M of N', 'Boolean attributes', 'class noise', 'computational tractability', 'normal distribution', 'supervised induction', 'conditional probability distribution']
['representation-oriented', 'process-oriented evaluation', 'generalization error', 'search process', 'rule induction', 'CN2 rule learner', 'high-dimensional domains', 'simpler models']
['Model selection', 'Bayesian', 'priors on error values', 'expected true error', 'classification learning', 'empirical minimizer', 'error distribution', 'PAC theory', 'VC-dimension', '10-fold cross-validation']
['machine learning algorithms', 'benchmark data sets', 'domain-specific parameters', 'error rate', 'optimistic bias', 'more parameters', 'fewer parameters', 'C4.5', 'FOIL', 'unbiased ranking experi